In [1]:
import os

In [2]:
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\Food Recommedor\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\Food Recommedor'

In [5]:
from  pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    training_data:Path
    updated_base_model_path:Path
    saved_features:Path
    



In [6]:
from src.Food_Recommendation.constants import *
from src.Food_Recommendation.utils.common import read_yaml,create_directories,feature_extraction,save_object
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def get_training_config(self) -> TrainingConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,'images')
        
        create_directories([Path(training.root_dir)])
        training_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            saved_features=Path(training.saved_features))
        return training_config
    


         

In [8]:
import os
import tensorflow as tf
import time
import urllib.request as request
from zipfile import ZipFile

In [9]:
class Training:
    def __init__(self,config:TrainingConfig):
        self.config=config
        self.img_name=[]
    def complete_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    def extract_path(self):
        path=self.config.training_data
        for image_path in os.listdir(path):
            self.img_name.append(os.path.join(path,image_path))
    

    def extract_feature(self):
        feature_list=[]
        for image in self.img_name:
            features=feature_extraction(image,self.model)
            feature_list.append(features)
        save_object(file_path=self.config.saved_features,obj=feature_list)
        
        
        

    


In [10]:
try:
    config=ConfigurationManager()
    training_config=config.get_training_config()
    training=Training(config=training_config)
    training.complete_model()
    training.extract_path()
    training.extract_feature()
except Exception as e:
    raise e


[2024-01-23 22:39:32,435: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-23 22:39:32,437: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-23 22:39:32,438: INFO: common: created directory at: artifacts]
[2024-01-23 22:39:32,440: INFO: common: created directory at: artifacts\training]
[2024-01-23 22:39:34,457: WARNING: hdf5_format: No training configuration found in the save file, so the model was *not* compiled. Compile it manually.]
1/1 [==============================] - 0s 83ms/step
